## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,12,Vaini,TO,80.76,shower rain,-21.2000,-175.2000,Keleti Beach Resort
1,31,Butaritari,KI,82.36,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
2,37,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang
3,43,Lorengau,PG,80.78,few clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
4,63,Arraial Do Cabo,BR,82.36,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
5,69,Yulara,AU,82.24,clear sky,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
6,76,Karratha,AU,84.76,light rain,-20.7377,116.8463,ibis Styles Karratha
7,89,San Vicente,AR,85.50,light rain,-26.6167,-54.1333,Complejo Turístico Farelluk
8,99,Chuy,UY,86.83,light rain,-33.6971,-53.4616,Nuevo Hotel Plaza
9,105,Samarai,PG,80.87,overcast clouds,-10.6167,150.6667,Nuli Sapi


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
vacation_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.

locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Acapulco"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Acapulco"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Lazaro Cardenas"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "El Colomo"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Ixtapa"]

In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
# Had difficulty with code erring out. Used code from CMOCONNOR17 "Vacation_Itinerary.ipynb" through GitHub.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]
vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,175,Acapulco,MX,84.16,clear sky,16.8634,-99.8901,HS Hotsson Smart Acapulco


In [7]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacation_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')
fig.add_layer(vacation_itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,175,Acapulco,MX,84.16,clear sky,16.8634,-99.8901,HS Hotsson Smart Acapulco
1,476,Lazaro Cardenas,MX,85.12,clear sky,17.9583,-102.2000,quinta antigua
2,550,El Colomo,MX,86.13,clear sky,19.0500,-104.2500,Hotel Luna del Mar
3,212,Ixtapa,MX,88.63,clear sky,20.7000,-105.2000,Hotel Plaza Providencia


In [9]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
itinerary_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [10]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=1000, point_radius=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=itinerary_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))